In [1]:
# Importation des librairies
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import hex2color

import seaborn as sns
import plotly.express as px

import datetime as dt
import scipy.stats as st
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
conf_graph = {
  'set_style': {
    'axes.spines.bottom': False,
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.top': False,
    'axes.grid': True,
    'grid.color': '.9'},
  'title_style': {
    'loc': 'left',
    'fontsize': 20,
    'fontweight': 'bold',
    'color': '#000000'},
  'label_style': {
    'fontsize': 15,
    'fontweight': 'bold',
    'color': '#000000'},
  'tick_style': {
    'fontsize': 10,
    'fontweight': 'bold',
    'color': '#000000'},
  'rotation': {
    'rotation': '20',
    'horizontalalignment': 'right'},
  'legend_style': {
      'weight': 'bold',
      'size': 10},
  'dscolors': {
    'pink': '#DA9BEA',
    'lilac': '#959FF5',
    'blue': '#4FD7DB',
    'green': '#89F594',
    'yellow': '#EBEA8A',
    'orange': '#FAC19D'}}

sns.set_style("white", conf_graph['set_style'])
palette = sns.color_palette(list(conf_graph['dscolors'].values()))


def NonLinCdict(steps, hexcol_array):
    """Fonction pour faire un mapping de couleur pour seaborn."""
    cdict = {'yellow': (), 'green': (), 'blue': ()}
    for s, hexcol in zip(steps, hexcol_array):
        rgb = hex2color(hexcol)
        cdict['yellow'] = cdict['yellow'] + ((s, rgb[0], rgb[0]),)
        cdict['green'] = cdict['green'] + ((s, rgb[1], rgb[1]),)
        cdict['blue'] = cdict['blue'] + ((s, rgb[2], rgb[2]),)

    return cdict


cdict = NonLinCdict([0, 1], palette)
cm = LinearSegmentedColormap('MDM', cdict)

# Importation des données

##### Fichiers xlsl fournis par OC

Cabinet de consultant spécialisé dans la transformation digitale des entreprises. 
Le cabinet compte déjà plus de 150 salariés et est en plein développement. 
Dans ce contexte économique, le recrutement de consultants expérimentés devient un véritable enjeu stratégique.

In [3]:
# Importation des données
df_infos=pd.read_excel('info_pro.xlsx')
df_remu=pd.read_excel('remunerations.xlsx')
df_salaries=pd.read_excel('salaries.xlsx')

In [4]:
# Afficher le résumé complet de df_infos
df_infos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id_salarié                 256 non-null    object 
 1   Ancienneté_an              256 non-null    float64
 2   Distance domicile/Travail  256 non-null    int64  
 3   Service                    256 non-null    object 
 4   Work_accident              256 non-null    object 
 5   Niveau de satisfaction     256 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 12.1+ KB


In [5]:
# Affichage de la 1ère ligne de df_infos
df_infos.head(1)

,id_salarié,Ancienneté_an,Distance domicile/Travail,Service,Work_accident,Niveau de satisfaction
0,16000206-8278,12.6,59,Marketing,Non,67


In [6]:
# Afficher le résumé complet de df_remu
df_remu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_salarié            256 non-null    object 
 1   Contrat               256 non-null    object 
 2   Durée hebdo           256 non-null    int64  
 3   Salaire base mensuel  256 non-null    float64
 4   %variable_moyen       256 non-null    int64  
 5   Augmentation          241 non-null    float64
 6   Promotion             241 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 14.1+ KB


In [7]:
# Affichage de la 1ère ligne de df_remu
df_remu.head(1)

,id_salarié,Contrat,Durée hebdo,Salaire base mensuel,%variable_moyen,Augmentation,Promotion
0,16990924-2829,CDI,35,9873.5,13,0.0,0.0


In [8]:
# Afficher le résumé complet de df_salaries
df_salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_salarié      256 non-null    object        
 1   Sexe            256 non-null    object        
 2   Prénom/Nom      256 non-null    object        
 3   Telephone       256 non-null    int64         
 4   Date_naissance  256 non-null    datetime64[ns]
 5   Etat Civil      174 non-null    object        
 6   Enfants         256 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 14.1+ KB


In [9]:
# Affichage de la 1ère ligne de df_salaries
df_salaries.head(1)

,id_salarié,Sexe,Prénom/Nom,Telephone,Date_naissance,Etat Civil,Enfants
0,16920105-1209,F,Solenne Delannoy,607958599,1955-07-07,Célibataire,1


In [10]:
# Jointure entre les 3 datasets
df = df_remu.merge(df_infos, how='left', on='id_salarié').copy()
df = df.merge(df_salaries, how='left', on='id_salarié')

In [11]:
# Afficher le résumé complet de df, notre dataset principal
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 255
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id_salarié                 256 non-null    object        
 1   Contrat                    256 non-null    object        
 2   Durée hebdo                256 non-null    int64         
 3   Salaire base mensuel       256 non-null    float64       
 4   %variable_moyen            256 non-null    int64         
 5   Augmentation               241 non-null    float64       
 6   Promotion                  241 non-null    float64       
 7   Ancienneté_an              256 non-null    float64       
 8   Distance domicile/Travail  256 non-null    int64         
 9   Service                    256 non-null    object        
 10  Work_accident              256 non-null    object        
 11  Niveau de satisfaction     256 non-null    int64         
 12  Sexe    

In [12]:
df.isna().sum()

id_salarié                    0
Contrat                       0
Durée hebdo                   0
Salaire base mensuel          0
%variable_moyen               0
Augmentation                 15
Promotion                    15
Ancienneté_an                 0
Distance domicile/Travail     0
Service                       0
Work_accident                 0
Niveau de satisfaction        0
Sexe                          0
Prénom/Nom                    0
Telephone                     0
Date_naissance                0
Etat Civil                   82
Enfants                       0
dtype: int64

# Nettoyage des données

In [13]:
# Vérification des valeurs manquantes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 255
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id_salarié                 256 non-null    object        
 1   Contrat                    256 non-null    object        
 2   Durée hebdo                256 non-null    int64         
 3   Salaire base mensuel       256 non-null    float64       
 4   %variable_moyen            256 non-null    int64         
 5   Augmentation               241 non-null    float64       
 6   Promotion                  241 non-null    float64       
 7   Ancienneté_an              256 non-null    float64       
 8   Distance domicile/Travail  256 non-null    int64         
 9   Service                    256 non-null    object        
 10  Work_accident              256 non-null    object        
 11  Niveau de satisfaction     256 non-null    int64         
 12  Sexe    

In [14]:
# Visualisation des valeurs manquantes
df.loc[df['Augmentation'].isna()]

,id_salarié,Contrat,Durée hebdo,Salaire base mensuel,%variable_moyen,Augmentation,Promotion,Ancienneté_an,Distance domicile/Travail,Service,Work_accident,Niveau de satisfaction,Sexe,Prénom/Nom,Telephone,Date_naissance,Etat Civil,Enfants
26,16891203-4397,CDD,32,2496.0000,21,NaN,NaN,1.3,43,Consultant,Oui,6,F,Micheline Poussin,693314366,1967-10-08,Célibataire,0
46,16810820-7468,CDD,35,3168.3167,10,NaN,NaN,1.4,42,Consultant,Non,18,F,Bérengère Levasseur,614612887,1971-08-30,Marié,3
48,16810711-6330,CDD,35,1989.8667,5,NaN,NaN,1.4,67,Marketing,Non,19,H,Anatole Girault,683124664,1980-11-25,NaN,0
52,16790704-0179,CDD,35,3185.0000,3,NaN,NaN,1.7,60,Commercial,Oui,58,F,Maud Rousselle,637914433,1971-12-31,Marié,3
56,16770802-4604,CDD,35,3033.3333,8,NaN,NaN,0.4,17,Consultant,Oui,24,F,Christelle Tomas,629546308,1956-03-17,Marié,2
66,16740208-0084,CDD,35,2881.6667,27,NaN,NaN,1.9,15,Commercial,Oui,76,F,Alberte Mossé,657984257,1964-01-26,Célibataire,1
68,16730918-6273,CDD,35,2426.6667,21,NaN,NaN,0.6,14,Marketing,Oui,70,F,Anne Beaumanoir,670650694,1967-01-03,Marié,1
82,16690714-7836,CDD,35,3640.0000,6,NaN,NaN,1.1,9,Marketing,Oui,96,F,Nancy Delacroix,682388988,1971-03-25,NaN,1
86,16670913-0147,CDD,35,3201.6833,38,NaN,NaN,0.8,8,Compta Finances,Oui,45,F,Jessica Brousseau,698491351,1990-05-20,Célibataire,1
125,16540728-0337,CDD,35,2123.3333,39,NaN,NaN,1.8,6,Compta Finances,Non,84,F,Nikita Brosseau,660672943,1959-12-26,NaN,1


In [15]:
# Vérifier la cohérence Nan = CDD
df.loc[df["Contrat"] == "CDD",:]

,id_salarié,Contrat,Durée hebdo,Salaire base mensuel,%variable_moyen,Augmentation,Promotion,Ancienneté_an,Distance domicile/Travail,Service,Work_accident,Niveau de satisfaction,Sexe,Prénom/Nom,Telephone,Date_naissance,Etat Civil,Enfants
4,16960906-7989,CDD,35,2848.3000,15,1.0,0.0,3.3,15,R&D,Oui,81,H,Fabien Cerf,626873728,1983-03-04,NaN,0
26,16891203-4397,CDD,32,2496.0000,21,NaN,NaN,1.3,43,Consultant,Oui,6,F,Micheline Poussin,693314366,1967-10-08,Célibataire,0
46,16810820-7468,CDD,35,3168.3167,10,NaN,NaN,1.4,42,Consultant,Non,18,F,Bérengère Levasseur,614612887,1971-08-30,Marié,3
48,16810711-6330,CDD,35,1989.8667,5,NaN,NaN,1.4,67,Marketing,Non,19,H,Anatole Girault,683124664,1980-11-25,NaN,0
52,16790704-0179,CDD,35,3185.0000,3,NaN,NaN,1.7,60,Commercial,Oui,58,F,Maud Rousselle,637914433,1971-12-31,Marié,3
56,16770802-4604,CDD,35,3033.3333,8,NaN,NaN,0.4,17,Consultant,Oui,24,F,Christelle Tomas,629546308,1956-03-17,Marié,2
66,16740208-0084,CDD,35,2881.6667,27,NaN,NaN,1.9,15,Commercial,Oui,76,F,Alberte Mossé,657984257,1964-01-26,Célibataire,1
68,16730918-6273,CDD,35,2426.6667,21,NaN,NaN,0.6,14,Marketing,Oui,70,F,Anne Beaumanoir,670650694,1967-01-03,Marié,1
82,16690714-7836,CDD,35,3640.0000,6,NaN,NaN,1.1,9,Marketing,Oui,96,F,Nancy Delacroix,682388988,1971-03-25,NaN,1
86,16670913-0147,CDD,35,3201.6833,38,NaN,NaN,0.8,8,Compta Finances,Oui,45,F,Jessica Brousseau,698491351,1990-05-20,Célibataire,1


In [16]:
# Remplacement des NaN par 0.0 elles ne concernent que les CDD
df=df.fillna(0.0)

In [17]:
# Vérification des doublons
df.groupby('id_salarié').count()

,Contrat,Durée hebdo,Salaire base mensuel,%variable_moyen,Augmentation,Promotion,Ancienneté_an,Distance domicile/Travail,Service,Work_accident,Niveau de satisfaction,Sexe,Prénom/Nom,Telephone,Date_naissance,Etat Civil,Enfants
id_salarié,,,,,,,,,,,,,,,,,
16000206-8278,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
16011205-0331,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
16020327-1465,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
16020701-3376,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
16020725-8401,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16960906-7989,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
16970415-1050,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
16981206-3817,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [18]:
# Vérification des outliers
df.describe().round(2)

,Durée hebdo,Salaire base mensuel,%variable_moyen,Augmentation,Promotion,Ancienneté_an,Distance domicile/Travail,Niveau de satisfaction,Telephone,Enfants
count,256.00,256.00,256.00,256.00,256.00,256.00,256.00,256.00,2.560000e+02,256.00
mean,33.87,5326.53,20.03,0.49,0.49,13.16,38.43,50.69,6.511928e+08,1.46
std,3.20,2505.49,11.53,0.50,0.50,7.02,19.83,27.84,2.915357e+07,1.12
min,24.00,1059.76,0.00,0.00,0.00,0.30,3.00,1.00,6.003126e+08,0.00
25%,35.00,3111.92,10.00,0.00,0.00,7.38,22.00,30.00,6.251080e+08,0.00
50%,35.00,5084.62,19.00,0.00,0.00,12.75,40.50,50.50,6.514391e+08,1.00
75%,35.00,7407.02,30.00,1.00,1.00,19.30,57.00,73.00,6.783984e+08,2.00
max,35.00,9999.38,40.00,1.00,1.00,25.00,70.00,100.00,6.991239e+08,3.00


# Clarifier les données

In [19]:
# Calculer l'âge de chaque salarié à parti de sa DdN
def from_dob_to_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
df['Date_naissance']=df['Date_naissance'].apply(lambda x: from_dob_to_age(x))

# Clarifier le nom de certaines variables pour une meilleure compréhension
df.rename(columns={'Salaire base mensuel': 'Salaire_brut_mens'}, inplace=True)
df.rename(columns={'Durée hebdo': 'Temps_trav'}, inplace=True)
df.rename(columns={'Date_naissance': 'Age'}, inplace=True)

# Clarifier les modalités de certaines colonnes
df.loc[df["Temps_trav"] == 35, "Temps_trav"] = "Temps complet"
df.loc[df["Temps_trav"].isin([24,28,32]), "Temps_trav"] = "Temps partiel"
df.loc[df["Sexe"] == "H", "Sexe"] = "Homme"
df.loc[df["Sexe"] == "F", "Sexe"] = "Femme"

# Ne garder que les colonnes pertinentes pour l'étude (retrait données sensibles et inutiles)
df = df.drop(columns=['Distance domicile/Travail','%variable_moyen','Work_accident',
                      'Niveau de satisfaction','Prénom/Nom','Telephone','Etat Civil',
                      'Augmentation','Promotion','Enfants'])

In [20]:
df.to_csv('df_oc.csv', index = False) #dans Python